<a href="https://colab.research.google.com/github/augustolnb/electrical_engineering/blob/main/malha_aterramento_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
import numpy as np
import pandas as pd

#### Cálculo do condutor

In [161]:
# Dimensionamento dos condutores

# t_amb -> temperatura ambiente
# t_max_malha -> temperatura máxima do cabo da malha
# t_max_lig -> temperatura máxima do cabo de ligação
# s_cond -> seção transversal do condutor [mm²]
# i_falta -> corrente de falta
# t_falta -> tempo de falta
#

t_falta = 0.5
t_amb = 30
t_max_malha = 450
t_max_lig = 450
i_falta = 3200
i_malha = 1220

# Calculo do condutor da malha
#i_falta = float(input("Corrente máxima de falta [A]: "))
# Corrente de malha = 60% corrente de falta

# calculo da corrente de falta, totalmente diferente do que consta no pdf
#i_falta = 226.53 * s_cond_malha * np.sqrt((1/t_falta) * np.log((t_max_malha - t_amb) / (234 + t_amb) + 1))
#i_malha = 0.6 * i_falta

s_cond_lig = i_falta / (226.53 * np.sqrt((1/t_falta) * np.log((t_max_malha - t_amb) / (234 + t_amb) + 1)))
s_cond_malha = i_malha / (226.53 * np.sqrt((1/t_falta) * np.log((t_max_malha - t_amb) / (234 + t_amb) + 1)))

# Adicionar condicional para setar valores padronizados de condutores
'''
if(s_cond_lig < 35):
    s_cond_lig = 35
    print("Valor mínimo prático é de 35mm²")
if(s_cond_malha < 35):
    s_cond_malha = 35
    print("Valor mínimo prático é de 35mm²")
'''

# Diâmetro do condutor em metros [d >= 0.25.h]
diametro_cond = (np.sqrt(4 * s_cond_lig / np.pi)) / 1000


print("Condutor de ligação: {} mm²".format(np.round(s_cond_lig, 2)))
print("\nCondutor de malha: {} mm²".format(np.round(s_cond_malha, 2)))
print("\nDiâmetro do condutor de ligação: {} m".format(np.round(diametro_cond, 5)))


Condutor de ligação: 10.24 mm²

Condutor de malha: 3.9 mm²

Diâmetro do condutor de ligação: 0.00361 m


#### Cálculo da Resistência da Malha

In [162]:
# Resistência de uma haste isolada
#
#   R = (rho_aparente/(np.pi*2*L)*np.ln(4*L/d))
#   
#   onde:
#   L               -> Comprimento da haste [m]
#   d               -> Diâmetro da haste [m]
#   rho_aparente    -> Resistividade aparente do solo [Ohm.m]
#   h               -> Profundidade da malha [m]
#   C_malha         -> Comprimento da malha [m]
#   L_malha         -> Largura da malha [m]
#   e_a             -> Espaçamento dos condutores no comprimento
#   e_b             -> Espaçamento dos condutores na largura
#   n_hastes        -> Numº de hastes [un]
#   comp_total_cond -> Comprimento total do condutores [m]

C_malha = 30
L_malha = 30
h = 0.6
L = 2.4
d = 0.025
rho_aparente = 411.8
e_a = 5 # Espaçamentos entre os condutores da malha, no comprimento
e_b = 5 # Espaçamentos entre os condutores da malha, na largura
e = 10 # Espaçamentos entre as hastes

area_total = C_malha * L_malha
n_a = int((C_malha/e_a) + 1)
n_b = int((L_malha/e_b) + 1)
comp_cond = C_malha * n_b + L_malha * n_a
#comp_total_hastes = n_hastes * L
#comp_total = comp_total_cond + comp_total_hastes

# Resistência de uma haste individual - Diagonal principal da matriz de resistências
R = (rho_aparente/(np.pi*2*L)) * np.log(4*L/d)

In [163]:
# Gera vetor de hastes, o numº de cada haste é a chave para um 
# Dicionário que contém as coordenadas daquela haste
hastes = []
cont = 0
coordenadas = dict()
k = 0

for i in range(n_b):
    hastes.append(np.round(np.arange(0, C_malha+1 , e), 1))

n_hastes = len(hastes) * len(np.arange(0, C_malha+1 , e))

# Comprimento total dos condutores
comp_hastes = n_hastes * L
comp_total = comp_cond + comp_hastes

while k < n_b:
    for j in range(n_a):
        coordenadas[cont] = j*e, k*e    # associa a haste à sua respectiva coordenada
        cont += 1
    k += 1

In [164]:
# Cálculo da matriz de resistências 
# Matriz triangular, da diagonal para baixo ela repete o padrão da parte superior
r = np.zeros((n_hastes,n_hastes), dtype=float) # matriz de resistências

for i in range(n_hastes): # laço para percorrer linhas da matriz de coordenadas
    for j in range(n_hastes): # laço para percorrer colunas da matriz de coordenadas
        if(i==j): 
            r[i,j] = np.round(R,2)
            #r_ind[i] += np.round(R,2)
            continue
        elif(i < j):
            haste_ref = coordenadas[i]
            hastezinha = coordenadas[j]
            x = np.sqrt((haste_ref[0] - hastezinha[0])**2 + (haste_ref[1] - hastezinha[1])**2)
            b = np.sqrt(x**2 + L**2)

            if(x**2 == (b-L)**2):
                #r[i,j] = (411.8 / (4*np.pi*L)) * (np.log(((b+L)**2 - x**2) / (0.0001)))
                continue
            else:
                r[i,j] = np.round((411.8 / (4*np.pi*L)) * (np.log(((b+L)**2 - x**2) / (x**2 - (b-L)**2))), 2)
                r[j, i] = r[i,j]
                

In [165]:
# Cálculo da resistência equivalente das hastes
r_ind = np.zeros(n_hastes)
r_eq = 0

for i in range(n_hastes): # laço para percorrer linhas da matriz de coordenadas
    for j in range(n_hastes): # laço para percorrer colunas da matriz de coordenadas
        r_ind[i] += r[i,j]

    r_eq += 1/r_ind[i]

#print("\nResistência equivalente das hastes: {} Ohm".format(np.round(r_eq,4)))

# Resistência total da malha
r_total = rho_aparente * ((1/comp_total) + ((1/(np.sqrt(20*C_malha*L_malha)))*(1+(1/(1+(h*(np.sqrt(20/(C_malha*L_malha)))))))))
print("\nResistência final [malha + hastes]: {} Ohm".format(np.round(r_total,2)))
print("Numº de hastes: {}".format(n_hastes))

#with open("log_malha_aterramento.txt", "a", encoding="utf-8") as f:
#    read_data = f.write('ea\teb\te\tNumº de hastes\t\tR. da malha\t Pot. de Toque Calc.\tPot. de Toque Max.\n')
#    read_data = f.write('{}\t{}\t{}\t {} \t\t {} \t {} \t\t {} \n'.format(e_a, e_b, e, n_hastes, np.round(r_total,2), v_malha, v_ma))



Resistência final [malha + hastes]: 6.73 Ohm
Numº de hastes: 28


In [ ]:
# Apresentação da tabela de resistências
import pandas as pd
df = pd.DataFrame(r)
display(df)
#df.to_excel("tabela_de_resistencias.xlsx")

#### Cálculo da Tensão de Toque

In [167]:
# Cálculo de coeficientes
rho_brita = 3000

N = np.sqrt(n_a / n_b)
ki = 0.656 + 0.172 * N
#kii = 1/((2*N)**(2/N)) # P/ malhas sem hastes ou com poucas hastes
kii = 1
kh = np.sqrt(1+(h)) #Quanto vale h0??
km = 1/(2*np.pi) * (np.log((e**2/(16*h*d)) + (((e+2*h)**2)/(8*e*d)) - (h/(4*d))) + (kii/kh) * np.log(8/(np.pi*(2*N-1))))

v_toque_malha = (rho_aparente*km*ki*i_malha) / (comp_cond + 1.15*comp_hastes)

v_toque_max = 116 + 0.174 * rho_brita / np.sqrt(t_falta)


#print("Tensão na malha:{} V".format(np.round(v_toque_malha,2)))
#print("Tensão máxima:\t{} V".format(np.round(v_toque_max,2)))


In [168]:
# Gravar log dos valores em um arquivo txt

with open("log_malha_aterramento.txt", "a", encoding="utf-8") as f:
    str1 = "\n\n##########################################################################################\n\n"
    f.write(str1)
    f.write('\n')
    f.write('ea\teb\te\tNumº de hastes\t\tR. da malha\t Pot. de Toque Calc.\tPot. de Toque Max.\n\n')
    f.write('{}\t{}\t{}\t {} \t\t {} \t {} \t\t {} \n'.format(e_a, e_b, e, n_hastes, np.round(r_total,2), np.round(v_malha,2), np.round(v_max,2)))


#### Cálculo da Tensão de Passo

In [169]:
# Coeficientes

kp = (1/np.pi) * ((1/(2*h)) + (1/(e+h)) + ((1/e) * (1-0.5**(N-2))))

v_passo_malha = (rho_aparente * kp * ki * i_malha) / comp_total
v_passo_max = (116 + 0.696*rho_brita) / np.sqrt(t_falta)

print('Comp. malha\tLarg. malha\tea\teb\te\tL da haste\tNº de hastes\tR. da malha\tPot. de Toque Calc.\tPot. de Toque Max.\tPot. de Passo Calc.\tPot. de Passo Max\n')
print("{} m\t\t{} m\t\t{} m\t{} m\t{} m\t{} m\t\t{} \t\t{} Ω\t\t{} V\t\t{} V\t\t{} V\t\t{} V\n".format(C_malha, L_malha, e_a, e_b, e, L, n_hastes, np.round(r_total,2), np.round(v_toque_malha,2), np.round(v_toque_max,2), np.round(v_passo_malha,2), np.round(v_passo_max,2)))
#print('\n\nPot. de Passo Calc.\tPot. de Passo Max.\n')
#print('{} V\t\t {} V\n'.format(np.round(v_passo_malha,2), np.round(v_passo_max,2)))
if(int(r_eq) < 10): print("\nResistência: ok")
else: print("\nResistência: Recalcular")
if(int(v_toque_malha) < int(v_toque_max)): print("\nPot. de Toque: ok")
else: print("\nPot. de Toque: Recalcular")
if(int(v_passo_malha) < int(v_passo_max)): print("\nPot. de Passo: ok")
else: print("\nPot. de Passo: Recalcular")

Comp. malha	Larg. malha	ea	eb	e	L da haste	Nº de hastes	R. da malha	Pot. de Toque Calc.	Pot. de Toque Max.	Pot. de Passo Calc.	Pot. de Passo Max

30 m		30 m		5 m	5 m	10 m	2.4 m		28 		6.73 Ω		918.49 V		854.22 V		224.95 V		3116.93 V


Resistência: ok

Pot. de Toque: Recalcular

Pot. de Passo: ok


##_Main()_

In [170]:
# função main()

if __name__ == '__main__':
    print("Teste")

Teste
